In [1]:
import sys
!{sys.executable} -m pip install -r requirements.txt

from tqdm import tqdm
import pandas as pd
import pprint
import project_helper
import lxml
from pymongo import MongoClient
import pymongo  # package for working with MongoDB
from bs4 import BeautifulSoup
import re
import datetime
# import pyspark
import Stocklist
import re
from requests import get
from collections import Counter 
import re, requests
import numpy as np
import sqlite3

You should consider upgrading via the '/Users/layalalzaydi/Desktop/Fillings/venv/bin/python -m pip install --upgrade pip' command.


In [160]:
cik_lookup = {
    'AMZN': '1018724',
'BMY': '0000014272',   
    'CNP': '0001130310',
    'CVX': '0000093410',
    'FL': '0000850209',
    'FRT': '0000034903',
    'HON': '0000773840'}

In [161]:
sec_api = project_helper.SecAPI()

In [162]:
%psource project_helper


In [163]:
def get_sec_data(cik, doc_type='',dateb='', owner='include', start=0, count=100,search_text=''):
    newest_pricing_data = pd.to_datetime('2020-07-01')

    
    
    rss_url = 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany' \
        '&CIK={}&type={}&dateb={}&owner={}&count={}&search_text={}&output=atom' \
        .format(cik, doc_type, dateb, owner , start ,count , search_text)
    
    print(rss_url)
    
    
    sec_data = sec_api.get(rss_url)
    feed = BeautifulSoup(sec_data.encode('ascii'), 'html').feed
    entries = [
        (
            
            entry.content.find('filing-date').getText())
        for entry in feed.find_all('entry', recursive=False)
        
        if pd.to_datetime(entry.content.find('filing-date').getText()) >= newest_pricing_data]
    print (entries)

    return entries
    


In [264]:
def get_sec_data_from_link(link):
    newest_pricing_data = pd.to_datetime('2020-07-01')


    
    sec_data = sec_api.get(link)
    feed = BeautifulSoup(sec_data.encode('ascii'), 'html').feed
    if feed is not None:
        entries = [
            (

                entry.content.find('filing-date').getText())
            for entry in feed.find_all('entry', recursive=False)

            if pd.to_datetime(entry.content.find('filing-date').getText()) >= newest_pricing_data]
        print (entries)

        return entries
    


In [235]:
def generate_link(cik, doc_type='',dateb='', owner='include', start=0, count=100,search_text=''):
    newest_pricing_data = pd.to_datetime('2020-07-01')

    rss_url = 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany' \
        '&CIK={}&type={}&dateb={}&owner={}&count={}&search_text={}&output=atom' \
        .format(cik, doc_type, dateb, owner , start ,count , search_text)
    return rss_url
    

In [246]:
cik_csv = pd.read_csv (r'/Users/layalalzaydi/Desktop/Fillings/cik.csv')
cik_csv = cik_csv.drop(columns=['Unnamed: 0'])
cik_csv['Link']= ""
cik_csv = cik_csv.set_index('ticker')
# loop pver the cik get it inot the function and put the return into the link column 
for CIK in cik_csv.cik:
    link = generate_link(CIK)
    cik_csv.loc[cik_csv['cik'] == CIK, 'Link'] = link
    

In [263]:
cik_csv.to_csv('allinfo.csv')
    

In [139]:
def get_sec_CIK_From_Ticker(tickers):
    
    cik_dict = {}
    
    for ticker in tickers:
        URL = 'http://www.sec.gov/cgi-bin/browse-edgar?CIK={}&Find=Search&owner=exclude&action=getcompany&output=atom'.format(ticker)

#         print(URL)

        sec_data = sec_api.get(URL)
        feed = BeautifulSoup(sec_data.encode('ascii'), 'html').feed
        
        print(type(feed))
        
        if feed is not None:
            result = feed.find('cik')

            print(len(result))
            print(result)

            if len(result):

                cik_dict[str(ticker).upper()] = result.get_text()

            print(result.get_text())
    

    return cik_dict

In [266]:
# example_ticker = 'AMZN'
sec_data = {}
for index in cik_csv.iterrows():
    print(index[1].Link)
    sec_data[index[0]] = get_sec_data_from_link(index[1].Link)

https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1090872&type=&dateb=&owner=include&count=0&search_text=100&output=atom
['2020-08-05', '2020-08-05', '2020-08-05', '2020-08-05', '2020-08-05', '2020-08-03', '2020-07-31', '2020-07-01']
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1675149&type=&dateb=&owner=include&count=0&search_text=100&output=atom
['2020-08-07', '2020-07-29', '2020-07-20', '2020-07-15', '2020-07-13', '2020-07-08', '2020-07-08', '2020-07-07', '2020-07-07', '2020-07-07']
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1555074&type=&dateb=&owner=include&count=0&search_text=100&output=atom
['2020-08-14', '2020-08-14']
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=706688&type=&dateb=&owner=include&count=0&search_text=100&output=atom
['2020-08-14', '2020-08-12', '2020-08-04', '2020-08-04', '2020-08-04', '2020-08-04', '2020-08-04', '2020-08-04', '2020-08-04', '2020-08-04']
https://www.sec.gov/cgi-bin/browse-edgar?

['2020-08-03', '2020-08-03', '2020-07-31', '2020-07-30', '2020-07-30', '2020-07-30', '2020-07-24', '2020-07-06']
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1703056&type=&dateb=&owner=include&count=0&search_text=100&output=atom
['2020-08-07', '2020-08-06', '2020-08-06', '2020-08-05', '2020-08-05', '2020-08-04', '2020-08-03', '2020-07-23', '2020-07-23', '2020-07-10']
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=2230&type=&dateb=&owner=include&count=0&search_text=100&output=atom
['2020-07-23', '2020-07-23', '2020-07-23']
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=2178&type=&dateb=&owner=include&count=0&search_text=100&output=atom
['2020-08-14', '2020-08-12', '2020-08-12', '2020-08-11', '2020-08-11', '2020-08-11', '2020-08-11', '2020-08-06', '2020-08-06']
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1002910&type=&dateb=&owner=include&count=0&search_text=100&output=atom
['2020-08-10', '2020-08-06', '2020-07-10', '20

['2020-08-14', '2020-08-14', '2020-08-14', '2020-08-14', '2020-08-11', '2020-08-03', '2020-08-03', '2020-07-09', '2020-07-06']
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=819793&type=&dateb=&owner=include&count=0&search_text=100&output=atom
['2020-08-07', '2020-07-30', '2020-07-29']
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1604738&type=&dateb=&owner=include&count=0&search_text=100&output=atom
['2020-08-10', '2020-08-10', '2020-07-31', '2020-07-31', '2020-07-24', '2020-07-13', '2020-07-08', '2020-07-02']
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1778114&type=&dateb=&owner=include&count=0&search_text=100&output=atom
['2020-07-16', '2020-07-07']
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1750&type=&dateb=&owner=include&count=0&search_text=100&output=atom
['2020-08-13', '2020-08-03', '2020-08-03', '2020-08-03', '2020-08-03', '2020-07-31', '2020-07-22', '2020-07-21', '2020-07-21', '2020-07-21']
https://www.sec

['2020-08-13', '2020-08-13', '2020-08-10', '2020-08-10', '2020-08-10', '2020-08-06']
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1129137&type=&dateb=&owner=include&count=0&search_text=100&output=atom
['2020-07-28', '2020-07-21', '2020-07-20', '2020-07-17']
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=820027&type=&dateb=&owner=include&count=0&search_text=100&output=atom
['2020-08-14', '2020-08-13', '2020-08-10', '2020-08-10', '2020-08-05', '2020-07-30', '2020-07-10', '2020-07-06', '2020-07-06', '2020-07-06']
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1411906&type=&dateb=&owner=include&count=0&search_text=100&output=atom
['2020-07-31', '2020-07-14', '2020-07-06', '2020-07-06', '2020-07-06', '2020-07-06', '2020-07-06']
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1533924&type=&dateb=&owner=include&count=0&search_text=100&output=atom
['2020-08-14', '2020-08-13', '2020-08-13', '2020-08-10', '2020-08-07', '2020-08-05'

['2020-08-11', '2020-07-29', '2020-07-29', '2020-07-17', '2020-07-17', '2020-07-17', '2020-07-14', '2020-07-14', '2020-07-14', '2020-07-14']
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1498068&type=&dateb=&owner=include&count=0&search_text=100&output=atom
['2020-08-10', '2020-08-10', '2020-07-09', '2020-07-09']
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1305168&type=&dateb=&owner=include&count=0&search_text=100&output=atom
['2020-08-05', '2020-08-04', '2020-07-16']
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1037676&type=&dateb=&owner=include&count=0&search_text=100&output=atom
['2020-08-10', '2020-07-28', '2020-07-28', '2020-07-06']
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1508478&type=&dateb=&owner=include&count=0&search_text=100&output=atom
['2020-08-12', '2020-08-12', '2020-07-10', '2020-07-06', '2020-07-06']
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1689662&type=&dateb=&owner=inclu

...

['2020-08-14', '2020-08-14', '2020-08-07', '2020-08-07', '2020-08-07', '2020-08-07', '2020-08-07', '2020-08-07', '2020-08-07', '2020-08-07']
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1104038&type=&dateb=&owner=include&count=0&search_text=100&output=atom
['2020-08-14', '2020-08-14', '2020-08-07', '2020-08-07', '2020-08-07', '2020-08-07', '2020-08-07', '2020-08-07', '2020-08-07', '2020-08-07']
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1657312&type=&dateb=&owner=include&count=0&search_text=100&output=atom
['2020-08-14', '2020-08-10', '2020-08-03', '2020-07-31', '2020-07-30', '2020-07-24', '2020-07-24', '2020-07-24', '2020-07-22', '2020-07-20']
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1361113&type=&dateb=&owner=include&count=0&search_text=100&output=atom
['2020-08-06', '2020-08-06', '2020-08-04', '2020-08-03', '2020-07-06', '2020-07-06', '2020-07-06', '2020-07-06', '2020-07-06']
https://www.sec.gov/cgi-bin/browse-edgar?action=

['2020-08-13', '2020-08-13', '2020-08-12', '2020-08-11', '2020-08-05', '2020-08-04', '2020-08-04', '2020-07-24', '2020-07-06', '2020-07-02']
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1169988&type=&dateb=&owner=include&count=0&search_text=100&output=atom
['2020-08-07', '2020-08-05', '2020-08-05', '2020-08-05', '2020-08-05', '2020-08-04', '2020-08-03', '2020-07-15', '2020-07-13']
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1030997&type=&dateb=&owner=include&count=0&search_text=100&output=atom
['2020-08-03']
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1770088&type=&dateb=&owner=include&count=0&search_text=100&output=atom
['2020-08-10', '2020-08-05', '2020-07-29', '2020-07-28', '2020-07-27', '2020-07-27', '2020-07-27', '2020-07-27', '2020-07-24', '2020-07-21']
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=908315&type=&dateb=&owner=include&count=0&search_text=100&output=atom
['2020-08-12', '2020-08-10', '2020-08-06'

['2020-08-07', '2020-07-29']
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1601669&type=&dateb=&owner=include&count=0&search_text=100&output=atom
['2020-08-07', '2020-07-29']
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1653247&type=&dateb=&owner=include&count=0&search_text=100&output=atom
['2020-08-10', '2020-08-06', '2020-08-06', '2020-08-06', '2020-07-15', '2020-07-10', '2020-07-09', '2020-07-06', '2020-07-02', '2020-07-02']
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1631574&type=&dateb=&owner=include&count=0&search_text=100&output=atom
['2020-08-10', '2020-08-10', '2020-08-10', '2020-08-10', '2020-07-02']
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=910679&type=&dateb=&owner=include&count=0&search_text=100&output=atom
['2020-07-30', '2020-07-30', '2020-07-30', '2020-07-29', '2020-07-29', '2020-07-29', '2020-07-27']
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=838875&type=&dateb=&owner=include

['2020-08-13', '2020-08-06', '2020-08-06', '2020-08-06', '2020-08-05', '2020-07-17', '2020-07-14', '2020-07-06']
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1513845&type=&dateb=&owner=include&count=0&search_text=100&output=atom
['2020-08-05', '2020-07-28']
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=108985&type=&dateb=&owner=include&count=0&search_text=100&output=atom
['2020-08-06', '2020-08-06', '2020-07-21', '2020-07-17', '2020-07-17', '2020-07-17', '2020-07-17', '2020-07-17', '2020-07-16', '2020-07-16']
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=716006&type=&dateb=&owner=include&count=0&search_text=100&output=atom
['2020-08-03', '2020-08-03', '2020-07-10', '2020-07-08', '2020-07-07']
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1121702&type=&dateb=&owner=include&count=0&search_text=100&output=atom
['2020-08-13', '2020-08-13', '2020-08-13', '2020-08-13', '2020-08-11', '2020-08-11', '2020-08-11', '2020-07-02',

['2020-08-06', '2020-08-06', '2020-07-30', '2020-07-30', '2020-07-30', '2020-07-30', '2020-07-28', '2020-07-28', '2020-07-24', '2020-07-20']


In [269]:
len(sec_data)

sec_data
import json
app_json = json.dumps(sec_data)

print(app_json)

with open('AllData.json', 'w') as json_file:
    json.dump(sec_data, json_file)

{"A": ["2020-08-05", "2020-08-05", "2020-08-05", "2020-08-05", "2020-08-05", "2020-08-03", "2020-07-31", "2020-07-01"], "AA": ["2020-08-07", "2020-07-29", "2020-07-20", "2020-07-15", "2020-07-13", "2020-07-08", "2020-07-08", "2020-07-07", "2020-07-07", "2020-07-07"], "AAMC": ["2020-08-14", "2020-08-14"], "AAN": ["2020-08-14", "2020-08-12", "2020-08-04", "2020-08-04", "2020-08-04", "2020-08-04", "2020-08-04", "2020-08-04", "2020-08-04", "2020-08-04"], "AAP": ["2020-08-13", "2020-07-23", "2020-07-23", "2020-07-08", "2020-07-08", "2020-07-08", "2020-07-08", "2020-07-08", "2020-07-08", "2020-07-08"], "AAT": ["2020-07-31", "2020-07-28", "2020-07-09"], "AAU": ["2020-08-11", "2020-08-07", "2020-08-06", "2020-08-05", "2020-07-23", "2020-07-01", "2020-07-01"], "AB": ["2020-08-11", "2020-08-10", "2020-07-23", "2020-07-23", "2020-07-13", "2020-07-06"], "ABB": ["2020-08-04", "2020-07-22", "2020-07-01"], "ABBV": ["2020-08-04", "2020-07-31", "2020-07-29", "2020-07-08", "2020-07-02", "2020-07-02", "2

In [143]:
cikamzn = get_sec_CIK_From_Ticker(x)

<class 'bs4.element.Tag'>
1
<cik>0001090872</cik>
0001090872
<class 'bs4.element.Tag'>
1
<cik>0001675149</cik>
0001675149
<class 'bs4.element.Tag'>
1
<cik>0001555074</cik>
0001555074
<class 'bs4.element.Tag'>
1
<cik>0000706688</cik>
0000706688
<class 'bs4.element.Tag'>
1
<cik>0001158449</cik>
0001158449
<class 'bs4.element.Tag'>
1
<cik>0001500217</cik>
0001500217
<class 'bs4.element.Tag'>
1
<cik>0001015647</cik>
0001015647
<class 'bs4.element.Tag'>
1
<cik>0000825313</cik>
0000825313
<class 'bs4.element.Tag'>
1
<cik>0001091587</cik>
0001091587
<class 'bs4.element.Tag'>
1
<cik>0001551152</cik>
0001551152
<class 'bs4.element.Tag'>
1
<cik>0001140859</cik>
0001140859
<class 'bs4.element.Tag'>
1
<cik>0001565025</cik>
0001565025
<class 'bs4.element.Tag'>
1
<cik>0001144980</cik>
0001144980
<class 'bs4.element.Tag'>
1
<cik>0000771497</cik>
0000771497
<class 'bs4.element.Tag'>
1
<cik>0001253986</cik>
0001253986
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'bs4.element.Tag'>
1


<class 'bs4.element.Tag'>
1
<cik>0001533924</cik>
0001533924
<class 'bs4.element.Tag'>
1
<cik>0001488139</cik>
0001488139
<class 'bs4.element.Tag'>
1
<cik>0001723128</cik>
0001723128
<class 'bs4.element.Tag'>
1
<cik>0000744825</cik>
0000744825
<class 'bs4.element.Tag'>
1
<cik>0001053507</cik>
0001053507
<class 'bs4.element.Tag'>
1
<cik>0001114446</cik>
0001114446
<class 'bs4.element.Tag'>
1
<cik>0001114446</cik>
0001114446
<class 'bs4.element.Tag'>
1
<cik>0001129137</cik>
0001129137
<class 'bs4.element.Tag'>
1
<cik>0000350698</cik>
0000350698
<class 'bs4.element.Tag'>
1
<cik>0001596532</cik>
0001596532
<class 'bs4.element.Tag'>
1
<cik>0001018840</cik>
0001018840
<class 'bs4.element.Tag'>
1
<cik>0001047884</cik>
0001047884
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'bs4.element.Tag'>
1
<cik>0001156039</cik>
0001156039
<class 'bs4.element.Tag'>
1
<cik>0001477845</cik>
0001477845
<class 'bs4.element.Tag'>
1
<cik>0001379400</cik>
0001379400
<class 'bs4.element.Tag'>
1


<class 'bs4.element.Tag'>
1
<cik>0001169770</cik>
0001169770
<class 'NoneType'>
<class 'NoneType'>
<class 'bs4.element.Tag'>
1
<cik>0001001290</cik>
0001001290
<class 'bs4.element.Tag'>
1
<cik>0001690437</cik>
0001690437
<class 'bs4.element.Tag'>
1
<cik>0001282648</cik>
0001282648
<class 'bs4.element.Tag'>
1
<cik>0000010456</cik>
0000010456
<class 'bs4.element.Tag'>
1
<cik>0001070235</cik>
0001070235
<class 'bs4.element.Tag'>
1
<cik>0000913059</cik>
0000913059
<class 'bs4.element.Tag'>
1
<cik>0001160330</cik>
0001160330
<class 'bs4.element.Tag'>
1
<cik>0001379785</cik>
0001379785
<class 'bs4.element.Tag'>
1
<cik>0001160330</cik>
0001160330
<class 'bs4.element.Tag'>
1
<cik>0001137392</cik>
0001137392
<class 'bs4.element.Tag'>
1
<cik>0001167467</cik>
0001167467
<class 'bs4.element.Tag'>
1
<cik>0001171264</cik>
0001171264
<class 'bs4.element.Tag'>
1
<cik>0001493683</cik>
0001493683
<class 'bs4.element.Tag'>
1
<cik>0001654795</cik>
0001654795
<class 'bs4.element.Tag'>
1
<cik>0000842180</ci

<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'bs4.element.Tag'>
1
<cik>0001784797</cik>
0001784797
<class 'NoneType'>
<class 'bs4.element.Tag'>
1
<cik>0000010048</cik>
0000010048
<class 'bs4.element.Tag'>
1
<cik>0000079282</cik>
0000079282
<class 'bs4.element.Tag'>
1
<cik>0000014846</cik>
0000014846
<class 'bs4.element.Tag'>
1
<cik>0001581068</cik>
0001581068
<class 'bs4.element.Tag'>
1
<cik>0001748824</cik>
0001748824
<class 'bs4.element.Tag'>
1
<cik>0001027552</cik>
0001027552
<class 'bs4.element.Tag'>
1
<cik>0001471055</cik>
0001471055
<class 'bs4.element.Tag'>
1
<cik>0001089094</cik>
0001089094
<class 'bs4.element.Tag'>
1
<cik>0001181024</cik>
0001181024
<class 'bs4.element.Tag'>
1
<cik>0001748824</cik>
0001748824
<class 'bs4.element.Tag'>
1
<cik>0001486298</cik>
0001486298
<class 'bs4.element.Tag'>
1
<cik>0001621434</cik>
0001621434
<class 'bs4.element.Tag'>
1
<cik>0001698287</cik>
0001698287
<class 'bs4.element.Tag'>
1
<cik>0001616678</cik>
0

<class 'NoneType'>
<class 'NoneType'>
<class 'bs4.element.Tag'>
1
<cik>0001631569</cik>
0001631569
<class 'bs4.element.Tag'>
1
<cik>0000313927</cik>
0000313927
<class 'bs4.element.Tag'>
1
<cik>0000019584</cik>
0000019584
<class 'bs4.element.Tag'>
1
<cik>0001364954</cik>
0001364954
<class 'bs4.element.Tag'>
1
<cik>0001046311</cik>
0001046311
<class 'bs4.element.Tag'>
1
<cik>0001117795</cik>
0001117795
<class 'bs4.element.Tag'>
1
<cik>0001571776</cik>
0001571776
<class 'NoneType'>
<class 'NoneType'>
<class 'bs4.element.Tag'>
1
<cik>0000845379</cik>
0000845379
<class 'bs4.element.Tag'>
1
<cik>0001730346</cik>
0001730346
<class 'bs4.element.Tag'>
1
<cik>0000897429</cik>
0000897429
<class 'bs4.element.Tag'>
1
<cik>0001132924</cik>
0001132924
<class 'bs4.element.Tag'>
1
<cik>0001113866</cik>
0001113866
<class 'bs4.element.Tag'>
1
<cik>0001766502</cik>
0001766502
<class 'bs4.element.Tag'>
1
<cik>0001723089</cik>
0001723089
<class 'bs4.element.Tag'>
1
<cik>0001739940</cik>
0001739940
<class 'b

...

<class 'bs4.element.Tag'>
1
<cik>0001684693</cik>
0001684693
<class 'bs4.element.Tag'>
1
<cik>0000849869</cik>
0000849869
<class 'bs4.element.Tag'>
1
<cik>0001032033</cik>
0001032033
<class 'NoneType'>
<class 'bs4.element.Tag'>
1
<cik>0001484565</cik>
0001484565
<class 'bs4.element.Tag'>
1
<cik>0001023459</cik>
0001023459
<class 'bs4.element.Tag'>
1
<cik>0001418076</cik>
0001418076
<class 'bs4.element.Tag'>
1
<cik>0001615219</cik>
0001615219
<class 'bs4.element.Tag'>
1
<cik>0001390478</cik>
0001390478
<class 'bs4.element.Tag'>
1
<cik>0001053092</cik>
0001053092
<class 'bs4.element.Tag'>
1
<cik>0000916907</cik>
0000916907
<class 'bs4.element.Tag'>
1
<cik>0001038773</cik>
0001038773
<class 'bs4.element.Tag'>
1
<cik>0001375365</cik>
0001375365
<class 'bs4.element.Tag'>
1
<cik>0000898770</cik>
0000898770
<class 'bs4.element.Tag'>
1
<cik>0000922612</cik>
0000922612
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'bs4.element.Tag'>
1
<cik>0000811808</cik>
0000811808
<class 'b

<class 'bs4.element.Tag'>
1
<cik>0001010612</cik>
0001010612
<class 'bs4.element.Tag'>
1
<cik>0000817720</cik>
0000817720
<class 'bs4.element.Tag'>
1
<cik>0001408278</cik>
0001408278
<class 'bs4.element.Tag'>
1
<cik>0001610950</cik>
0001610950
<class 'bs4.element.Tag'>
1
<cik>0000095953</cik>
0000095953
<class 'bs4.element.Tag'>
1
<cik>0000864240</cik>
0000864240
<class 'bs4.element.Tag'>
1
<cik>0001556263</cik>
0001556263
<class 'bs4.element.Tag'>
1
<cik>0001378453</cik>
0001378453
<class 'bs4.element.Tag'>
1
<cik>0001585583</cik>
0001585583
<class 'bs4.element.Tag'>
1
<cik>0001017303</cik>
0001017303
<class 'bs4.element.Tag'>
1
<cik>0000942126</cik>
0000942126
<class 'bs4.element.Tag'>
1
<cik>0001588084</cik>
0001588084
<class 'bs4.element.Tag'>
1
<cik>0001378453</cik>
0001378453
<class 'bs4.element.Tag'>
1
<cik>0001378453</cik>
0001378453
<class 'bs4.element.Tag'>
1
<cik>0001378453</cik>
0001378453
<class 'bs4.element.Tag'>
1
<cik>0001552670</cik>
0001552670
<class 'bs4.element.Tag'

<class 'bs4.element.Tag'>
1
<cik>0000893739</cik>
0000893739
<class 'bs4.element.Tag'>
1
<cik>0001679268</cik>
0001679268
<class 'bs4.element.Tag'>
1
<cik>0000704415</cik>
0000704415
<class 'bs4.element.Tag'>
1
<cik>0001758730</cik>
0001758730
<class 'bs4.element.Tag'>
1
<cik>0000100378</cik>
0000100378
<class 'bs4.element.Tag'>
1
<cik>0000795212</cik>
0000795212
<class 'bs4.element.Tag'>
1
<cik>0001644406</cik>
0001644406
<class 'bs4.element.Tag'>
1
<cik>0001644406</cik>
0001644406
<class 'bs4.element.Tag'>
1
<cik>0001459417</cik>
0001459417
<class 'bs4.element.Tag'>
1
<cik>0001581280</cik>
0001581280
<class 'bs4.element.Tag'>
1
<cik>0001770787</cik>
0001770787
<class 'bs4.element.Tag'>
1
<cik>0000025743</cik>
0000025743
<class 'bs4.element.Tag'>
1
<cik>0000097476</cik>
0000097476
<class 'bs4.element.Tag'>
1
<cik>0001289460</cik>
0001289460
<class 'bs4.element.Tag'>
1
<cik>0001300734</cik>
0001300734
<class 'bs4.element.Tag'>
1
<cik>0001537917</cik>
0001537917
<class 'bs4.element.Tag'

<class 'bs4.element.Tag'>
1
<cik>0001085243</cik>
0001085243
<class 'bs4.element.Tag'>
1
<cik>0001641489</cik>
0001641489
<class 'bs4.element.Tag'>
1
<cik>0001463972</cik>
0001463972
<class 'bs4.element.Tag'>
1
<cik>0001681348</cik>
0001681348
<class 'bs4.element.Tag'>
1
<cik>0000072444</cik>
0000072444
<class 'bs4.element.Tag'>
1
<cik>0001640266</cik>
0001640266
<class 'bs4.element.Tag'>
1
<cik>0000311094</cik>
0000311094
<class 'bs4.element.Tag'>
1
<cik>0000936528</cik>
0000936528
<class 'bs4.element.Tag'>
1
<cik>0001716770</cik>
0001716770
<class 'bs4.element.Tag'>
1
<cik>0000737468</cik>
0000737468
<class 'bs4.element.Tag'>
1
<cik>0001575793</cik>
0001575793
<class 'bs4.element.Tag'>
1
<cik>0001595761</cik>
0001595761
<class 'bs4.element.Tag'>
1
<cik>0001618921</cik>
0001618921
<class 'bs4.element.Tag'>
1
<cik>0001327811</cik>
0001327811
<class 'bs4.element.Tag'>
1
<cik>0000106040</cik>
0000106040
<class 'bs4.element.Tag'>
1
<cik>0000105132</cik>
0000105132
<class 'bs4.element.Tag'

<class 'bs4.element.Tag'>
1
<cik>0001318008</cik>
0001318008
<class 'bs4.element.Tag'>
1
<cik>0001305323</cik>
0001305323
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'bs4.element.Tag'>
1
<cik>0001621443</cik>
0001621443
<class 'bs4.element.Tag'>
1
<cik>0000846475</cik>
0000846475


In [147]:
# len(cikamzn)

# len(cikamzn.values())
len(cikamzn)

type(cikamzn)


cikamzn

{'A': '0001090872',
 'AA': '0001675149',
 'AAMC': '0001555074',
 'AAN': '0000706688',
 'AAP': '0001158449',
 'AAT': '0001500217',
 'AAU': '0001015647',
 'AB': '0000825313',
 'ABB': '0001091587',
 'ABBV': '0001551152',
 'ABC': '0001140859',
 'ABEV': '0001565025',
 'ABG': '0001144980',
 'ABM': '0000771497',
 'ABR': '0001253986',
 'ABT': '0000001800',
 'AC': '0001642122',
 'ACA': '0001739445',
 'ACB': '0001683541',
 'ACC': '0001283630',
 'ACCO': '0000712034',
 'ACEL': '0001698991',
 'ACH': '0001161611',
 'ACI': '0001646972',
 'ACM': '0000868857',
 'ACN': '0001467373',
 'ACP': '0001503290',
 'ACRE': '0001529377',
 'ACU': '0000002098',
 'ACV': '0001636289',
 'ACY': '0001036848',
 'ADC': '0000917251',
 'ADCT': '0001771910',
 'ADM': '0000007084',
 'ADNT': '0001670541',
 'ADS': '0001101215',
 'ADSW': '0001585790',
 'ADT': '0001703056',
 'ADX': '0000002230',
 'AE': '0000002178',
 'AEE': '0001002910',
 'AEF': '0000846676',
 'AEG': '0000769218',
 'AEL': '0001039828',
 'AEM': '0000002809',
 'AEO':

In [148]:
 dataCIK = pd.DataFrame(cikamzn.items(), columns=['ticker', 'cik'])

In [149]:
dataCIK

,ticker,cik
0,A,0001090872
1,AA,0001675149
2,AAMC,0001555074
3,AAN,0000706688
4,AAP,0001158449
...,...,...
5755,ZSAN,0001587221
5756,ZUMZ,0001318008
5757,ZVO,0001305323
5758,ZYNE,0001621443


In [156]:
dataCIK.set_index('ticker')

,cik
ticker,
A,0001090872
AA,0001675149
AAMC,0001555074
AAN,0000706688
AAP,0001158449
...,...
ZSAN,0001587221
ZUMZ,0001318008
ZVO,0001305323


In [151]:
csv_cik = dataCIK.to_csv('./cik.csv')


In [83]:
import json
app_json = json.dumps(cikamzn)
print(cikamzn)

with open('ticksvsciks2.json', 'w') as json_file:
    json.dump(cikamzn, json_file)

{'LASR': u'0001124796', 'SPR': u'0001364885', 'SPT': u'0001517375', 'SPH': u'0001005210', 'SPI': u'0001210618', 'SPN': u'0000886835', 'SPB': u'0000109177', 'SPE': u'0000897802', 'SPG': u'0001063761', 'XPO': u'0001166003', 'XPL': u'0000917225', 'PQG': u'0001708035', 'REML': u'0001053092', 'OKTA': u'0001660134', 'JNJ': u'0000200406', 'TRHC': u'0001651561', 'BEN': u'0000038777', 'BCYC': u'0001761612', 'SLDB': u'0001707502', 'BEP': u'0001533232', 'GRU': u'0000352960', 'VNTR': u'0001705682', 'GRX': u'0001391437', 'GRF': u'0000850027', 'GRC': u'0000042682', 'GRA': u'0001045309', 'GRN': u'0000312070', 'LPSN': u'0001102993', 'ITGR': u'0001114483', 'NOG': u'0001104485', 'NOA': u'0001368519', 'NOC': u'0001133421', 'HLI': u'0001302215', 'DYAI': u'0001213809', 'HLG': u'0001596964', 'HLF': u'0001180262', 'NOV': u'0001021860', 'NOW': u'0001373715', 'ATKR': u'0001666138', 'OLLI': u'0001639300', 'SYPR': u'0000864240', 'WSC': u'0001647088', 'BWXT': u'0001486957', 'FXNC': u'0000719402', 'ENZ': u'0000316

In [175]:
for k in sec_data.keys():
    sec_data[k] = Counter(sec_data[k])
    print(Counter(sec_data[k]))

Counter({'2020-08-05': 5, '2020-07-31': 2, '2020-08-10': 1, '2020-07-30': 1, '2020-07-02': 1})
Counter({'2020-08-06': 2, '2020-07-16': 2, '2020-07-06': 2, '2020-08-04': 1, '2020-07-30': 1, '2020-07-13': 1, '2020-07-02': 1})
Counter({'2020-07-30': 3, '2020-08-06': 2, '2020-07-31': 2, '2020-08-03': 1, '2020-07-20': 1, '2020-07-06': 1})
Counter({'2020-08-11': 4, '2020-08-13': 2, '2020-08-10': 2, '2020-08-06': 1, '2020-08-05': 1})
Counter({'2020-07-06': 5, '2020-08-03': 2, '2020-08-10': 1, '2020-07-23': 1, '2020-07-16': 1})
Counter({'2020-08-05': 2, '2020-08-10': 1})
Counter({'2020-08-03': 3, '2020-07-28': 2, '2020-08-05': 1, '2020-08-04': 1, '2020-07-30': 1, '2020-07-29': 1, '2020-07-24': 1})


In [13]:
DEFAULT_TICKERS = ['goog', 'aapl']
URL = 'http://www.sec.gov/cgi-bin/browse-edgar?CIK={}&Find=Search&owner=exclude&action=getcompany'
CIK_RE = re.compile(r'.*CIK=(\d{10}).*')

cik_dict = {}
for ticker in DEFAULT_TICKERS:
    results = CIK_RE.findall(get(URL.format(ticker)).content)
    if len(results):
        cik_dict[str(ticker).lower()] = str(results[0])
f = open('cik_dict', 'w')
dump(cik_dict, f)
f.close()

In [47]:
raw_fillings_by_ticker = {}

for ticker, data in sec_data.items():
    raw_fillings_by_ticker[ticker] = {}
    for index_url, file_type, file_date in tqdm(data, desc='Downloading {} Fillings'.format(ticker), unit='filling'):
        if (file_type == '10-K'):
            file_url = index_url.replace('-index.htm', '.txt').replace('.txtl', '.txt')            
            
            raw_fillings_by_ticker[ticker][file_date] = sec_api.get(file_url)


print('Example Document:\n\n{}...'.format(next(iter(raw_fillings_by_ticker[example_ticker].values()))[:1000]))

Example Document:

-----BEGIN PRIVACY-ENHANCED MESSAGE-----
Proc-Type: 2001,MIC-CLEAR
Originator-Name: webmaster@www.sec.gov
Originator-Key-Asymmetric:
 MFgwCgYEVQgBAQICAf8DSgAwRwJAW2sNKK9AVtBzYZmr6aGjlWyK3XmZv3dTINen
 TWSM7vrzLADbmYQaionwg5sDW3P6oaM5D3tdezXMm7z1T+B+twIDAQAB
MIC-Info: RSA-MD5,RSA,
 JheN9rrH0LADcPLNpdgic2Y+wa34llXqe750TowVzUFLd6IuuNQne2H0axAosYFz
 1Lmg9dJfEMxBnv2E6NCBrw==

<SEC-DOCUMENT>0001193125-05-047032.txt : 20050311
<SEC-HEADER>0001193125-05-047032.hdr.sgml : 20050311
<ACCEPTANCE-DATETIME>20050311062027
ACCESSION NUMBER:		0001193125-05-047032
CONFORMED SUBMISSION TYPE:	10-K
PUBLIC DOCUMENT COUNT:		9
CONFORMED PERIOD OF REPORT:	20041231
FILED AS OF DATE:		20050311
DATE AS OF CHANGE:		20050311

FILER:

	COMPANY DATA:	
		COMPANY CONFORMED NAME:			AMAZON COM INC
		CENTRAL INDEX KEY:			0001018724
		STANDARD INDUSTRIAL CLASSIFICATION:	RETAIL-CATALOG & MAIL-ORDER HOUSES [5961]
		IRS NUMBER:				911646860
		STATE OF INCORPORATION:			DE
		FISCAL YEAR END:			1231

	FILING VA

In [3]:
DEFAULT_TICKERS = ['goog', 'aapl','amzn']
URL = 'http://www.sec.gov/cgi-bin/browse-edgar?CIK={}&Find=Search&owner=exclude&action=getcompany'
CIK_RE = re.compile(r'.*CIK=(\d{10}).*')

cik_dict = {}
for ticker in DEFAULT_TICKERS:
    results = CIK_RE.findall(get(URL.format(ticker)).content)
    if len(results):
        cik_dict[str(ticker).lower()] = str(results[0])
f = open('cik_dict', 'w')
dump(cik_dict, f)
f.close()

In [129]:

def getCIKs(TICKERS):
    URL = 'http://www.sec.gov/cgi-bin/browse-edgar?CIK={}&Find=Search&owner=exclude&action=getcompany'
    CIK_RE = re.compile(r'.*CIK=(\d{10}).*')    
    cik_dict = {}
    for ticker in TICKERS:
        print(URL.format(ticker))
        f = sec_api.get(URL.format(ticker))
        
#         f = requests.get(URL.format(ticker), stream = True)
        results = CIK_RE.findall(f.text)
        print(cik_dict)
    
        if len(results):
            results[0] = int(re.sub('\.[0]*', '.', results[0]))
            cik_dict[str(ticker).upper()] = str(results[0])
    
    print(cik_dict)
    

getCIKs(['wmt','amzn','nflx'])

http://www.sec.gov/cgi-bin/browse-edgar?CIK=wmt&Find=Search&owner=exclude&action=getcompany


AttributeError: 'str' object has no attribute 'text'

In [27]:
y = getCIKs(x)

AttributeError: 'unicode' object has no attribute 'text'

In [302]:
def StockList():
        StocksController = Stocklist.NasdaqController(True)
        list_of_tickers = StocksController.getList()

        print(list_of_tickers)
        print(len(list_of_tickers))
        return list_of_tickers

In [303]:
x = StockList()

['A', 'AA', 'AAMC', 'AAN', 'AAP', 'AAT', 'AAU', 'AB', 'ABB', 'ABBV', 'ABC', 'ABEV', 'ABG', 'ABM', 'ABR', 'ABR$A', 'ABR$B', 'ABR$C', 'ABT', 'AC', 'ACA', 'ACB', 'ACC', 'ACCO', 'ACEL', 'ACH', 'ACI', 'ACM', 'ACN', 'ACND.U', 'ACP', 'ACRE', 'ACU', 'ACV', 'ACY', 'ADC', 'ADCT', 'ADM', 'ADNT', 'ADS', 'ADSW', 'ADT', 'ADX', 'AE', 'AEB', 'AEE', 'AEF', 'AEFC', 'AEG', 'AEL', 'AEL$A', 'AEL$B', 'AEM', 'AEO', 'AEP', 'AEP$B', 'AER', 'AES', 'AFB', 'AFC', 'AFG', 'AFGB', 'AFGC', 'AFGD', 'AFGH', 'AFI', 'AFL', 'AFT', 'AG', 'AGCO', 'AGD', 'AGE', 'AGI', 'AGM', 'AGM$A', 'AGM$C', 'AGM$D', 'AGM$E', 'AGM.A', 'AGO', 'AGO$B', 'AGO$E', 'AGO$F', 'AGR', 'AGRO', 'AGS', 'AGX', 'AHC', 'AHH', 'AHH$A', 'AHL$C', 'AHL$D', 'AHL$E', 'AHT', 'AHT$D', 'AHT$F', 'AHT$G', 'AHT$H', 'AHT$I', 'AI', 'AI$B', 'AI$C', 'AIC', 'AIF', 'AIG', 'AIG$A', 'AIG.W', 'AIM', 'AIN', 'AINC', 'AIO', 'AIR', 'AIRI', 'AIT', 'AIV', 'AIW', 'AIZ', 'AIZP', 'AJG', 'AJRD', 'AJX', 'AJXA', 'AKO.A', 'AKO.B', 'AKR', 'AL', 'AL$A', 'ALB', 'ALC', 'ALE', 'ALEX', 'ALG', 'A

In [304]:
x

['A',
 'AA',
 'AAMC',
 'AAN',
 'AAP',
 'AAT',
 'AAU',
 'AB',
 'ABB',
 'ABBV',
 'ABC',
 'ABEV',
 'ABG',
 'ABM',
 'ABR',
 'ABR$A',
 'ABR$B',
 'ABR$C',
 'ABT',
 'AC',
 'ACA',
 'ACB',
 'ACC',
 'ACCO',
 'ACEL',
 'ACH',
 'ACI',
 'ACM',
 'ACN',
 'ACND.U',
 'ACP',
 'ACRE',
 'ACU',
 'ACV',
 'ACY',
 'ADC',
 'ADCT',
 'ADM',
 'ADNT',
 'ADS',
 'ADSW',
 'ADT',
 'ADX',
 'AE',
 'AEB',
 'AEE',
 'AEF',
 'AEFC',
 'AEG',
 'AEL',
 'AEL$A',
 'AEL$B',
 'AEM',
 'AEO',
 'AEP',
 'AEP$B',
 'AER',
 'AES',
 'AFB',
 'AFC',
 'AFG',
 'AFGB',
 'AFGC',
 'AFGD',
 'AFGH',
 'AFI',
 'AFL',
 'AFT',
 'AG',
 'AGCO',
 'AGD',
 'AGE',
 'AGI',
 'AGM',
 'AGM$A',
 'AGM$C',
 'AGM$D',
 'AGM$E',
 'AGM.A',
 'AGO',
 'AGO$B',
 'AGO$E',
 'AGO$F',
 'AGR',
 'AGRO',
 'AGS',
 'AGX',
 'AHC',
 'AHH',
 'AHH$A',
 'AHL$C',
 'AHL$D',
 'AHL$E',
 'AHT',
 'AHT$D',
 'AHT$F',
 'AHT$G',
 'AHT$H',
 'AHT$I',
 'AI',
 'AI$B',
 'AI$C',
 'AIC',
 'AIF',
 'AIG',
 'AIG$A',
 'AIG.W',
 'AIM',
 'AIN',
 'AINC',
 'AIO',
 'AIR',
 'AIRI',
 'AIT',
 'AIV',
 'AIW',
 'AIZ',

In [305]:
def new_dataframe():
    
    StocksController = Stocklist.NasdaqController(True)
    list_of_tickers = StocksController.getList()

    
    index = pd.date_range(start='2020-07-01', end='today')
    df = pd.DataFrame(0,index = index, columns = list_of_tickers)
    df.tail()
    return df

In [216]:
# def populate(df):
#     for TKR , x in sec_data.items():
#         for i,num in x.items():
#             df.loc[i, TKR] = num
#     return df;

In [296]:
def populate_all(df):
    for TKR , x in data.items():
        if x:
            for i,num in x.items():
                df.loc[i, TKR] = num
    return df;

In [221]:
def append_TKR(newTRK_list, df):
    TKR_LIST = list(df.columns)
    TKR_LIST_diference = list(set(newTRK_list) - set(TKR_LIST))
    for i in TKR_LIST_diference:
        df.assign(ColName='i')
    return df

In [ ]:
def append_day():
    

In [306]:
#chnage the csv to a dictionary to pass it throught the counter 
import json
with open(r'/Users/layalalzaydi/Desktop/Fillings/AllData.json') as f:
    data = json.load(f)

for k in data.keys():
    data[k] = Counter(data[k])
    print(Counter(data[k]))
    
df_all = new_dataframe()

df_all_new = populate_all(df_all)



  
    
# cik_csv = pd.read_csv (r'/Users/layalalzaydi/Desktop/Fillings/cik.csv')
# cik_csv = cik_csv.drop(columns=['Unnamed: 0'])
# cik_csv['Link']= ""
# cik_csv = cik_csv.set_index('ticker')
# # loop pver the cik get it inot the function and put the return into the link column 
# for CIK in cik_csv.cik:
#     link = generate_link(CIK)
#     cik_csv.loc[cik_csv['cik'] == CIK, 'Link'] = link
    

Counter({'2020-08-05': 5, '2020-08-03': 1, '2020-07-31': 1, '2020-07-01': 1})
Counter({'2020-07-07': 3, '2020-07-08': 2, '2020-08-07': 1, '2020-07-29': 1, '2020-07-20': 1, '2020-07-15': 1, '2020-07-13': 1})
Counter({'2020-08-14': 2})
Counter({'2020-08-04': 8, '2020-08-14': 1, '2020-08-12': 1})
Counter({'2020-07-08': 7, '2020-07-23': 2, '2020-08-13': 1})
Counter({'2020-07-31': 1, '2020-07-28': 1, '2020-07-09': 1})
Counter({'2020-07-01': 2, '2020-08-11': 1, '2020-08-07': 1, '2020-08-06': 1, '2020-08-05': 1, '2020-07-23': 1})
Counter({'2020-07-23': 2, '2020-08-11': 1, '2020-08-10': 1, '2020-07-13': 1, '2020-07-06': 1})
Counter({'2020-08-04': 1, '2020-07-22': 1, '2020-07-01': 1})
Counter({'2020-07-02': 3, '2020-08-04': 1, '2020-07-31': 1, '2020-07-29': 1, '2020-07-08': 1})
Counter({'2020-08-05': 2, '2020-08-03': 2, '2020-07-10': 2, '2020-07-28': 1, '2020-07-21': 1, '2020-07-16': 1, '2020-07-07': 1})
Counter({'2020-07-30': 3, '2020-08-10': 2, '2020-07-13': 2, '2020-07-07': 2, '2020-08-12': 

Counter()
Counter({'2020-07-23': 2, '2020-07-22': 2, '2020-07-14': 2, '2020-08-11': 1, '2020-07-29': 1, '2020-07-17': 1, '2020-07-16': 1})
Counter({'2020-08-13': 1, '2020-07-15': 1, '2020-07-09': 1})
Counter({'2020-08-05': 1, '2020-08-04': 1})
Counter()
Counter({'2020-08-11': 1, '2020-07-09': 1})
Counter()
Counter()
Counter()
Counter()
Counter({'2020-07-29': 2, '2020-08-14': 1, '2020-08-10': 1})
Counter()
Counter()
Counter({'2020-08-14': 1, '2020-08-11': 1, '2020-07-22': 1, '2020-07-10': 1, '2020-07-01': 1})
Counter()
Counter({'2020-08-12': 1})
Counter({'2020-07-28': 4, '2020-08-13': 1, '2020-08-06': 1, '2020-07-30': 1, '2020-07-27': 1, '2020-07-23': 1, '2020-07-16': 1})
Counter({'2020-08-14': 9, '2020-08-13': 1})
Counter({'2020-08-14': 9, '2020-08-13': 1})
Counter({'2020-08-14': 9, '2020-08-13': 1})
Counter({'2020-08-14': 9, '2020-08-13': 1})
Counter({'2020-08-14': 9, '2020-08-13': 1})
Counter({'2020-08-14': 9, '2020-08-13': 1})
Counter({'2020-08-14': 9, '2020-08-13': 1})
Counter({'20

Counter({'2020-08-13': 2, '2020-08-12': 1, '2020-08-03': 1, '2020-07-30': 1, '2020-07-28': 1, '2020-07-13': 1, '2020-07-07': 1})
Counter({'2020-07-02': 3, '2020-08-06': 2, '2020-07-14': 2})
Counter({'2020-07-27': 2})
Counter({'2020-08-04': 1, '2020-07-31': 1, '2020-07-29': 1, '2020-07-28': 1})
Counter({'2020-07-15': 1})
Counter({'2020-08-13': 1, '2020-08-05': 1, '2020-07-28': 1})
Counter({'2020-07-14': 2, '2020-08-14': 1, '2020-08-10': 1, '2020-08-06': 1, '2020-07-02': 1})
Counter({'2020-08-06': 7, '2020-07-29': 2, '2020-08-03': 1})
Counter({'2020-07-17': 3, '2020-08-10': 1, '2020-07-27': 1})
Counter({'2020-08-06': 2, '2020-08-14': 1, '2020-08-11': 1})
Counter({'2020-07-01': 4, '2020-07-29': 2, '2020-07-31': 1, '2020-07-21': 1})
Counter()
Counter({'2020-07-28': 2, '2020-07-29': 1, '2020-07-07': 1})
Counter({'2020-08-14': 7, '2020-08-10': 1, '2020-08-07': 1, '2020-08-05': 1})
Counter({'2020-08-13': 2, '2020-07-31': 2, '2020-07-02': 1})
Counter({'2020-08-13': 1, '2020-08-05': 1, '2020-07

Counter({'2020-08-10': 2, '2020-08-04': 1, '2020-07-02': 1})
Counter({'2020-07-31': 2, '2020-07-20': 1, '2020-07-10': 1})
Counter({'2020-07-06': 2, '2020-08-12': 1, '2020-07-31': 1, '2020-07-20': 1, '2020-07-16': 1, '2020-07-07': 1, '2020-07-01': 1})
Counter({'2020-07-01': 2, '2020-08-06': 1, '2020-07-14': 1, '2020-07-10': 1, '2020-07-07': 1, '2020-07-06': 1, '2020-07-02': 1})
Counter({'2020-08-14': 1, '2020-08-12': 1, '2020-08-10': 1, '2020-08-06': 1, '2020-07-21': 1, '2020-07-13': 1, '2020-07-10': 1})
Counter({'2020-08-06': 1})
Counter({'2020-08-06': 1})
Counter({'2020-08-13': 2, '2020-08-14': 1, '2020-07-29': 1})
Counter({'2020-08-13': 2, '2020-08-14': 1, '2020-07-29': 1})
Counter({'2020-07-30': 3, '2020-07-28': 3, '2020-08-03': 2, '2020-08-05': 1, '2020-07-31': 1})
Counter({'2020-08-05': 1, '2020-07-22': 1, '2020-07-07': 1})
Counter({'2020-08-10': 2, '2020-08-11': 1, '2020-08-07': 1, '2020-07-30': 1, '2020-07-29': 1, '2020-07-28': 1, '2020-07-21': 1})
Counter({'2020-08-14': 2, '202

Counter({'2020-08-05': 2, '2020-07-02': 2})
Counter({'2020-08-05': 2, '2020-07-02': 2})
Counter({'2020-07-29': 1})
Counter({'2020-07-27': 3, '2020-07-31': 2, '2020-07-29': 1, '2020-07-24': 1, '2020-07-23': 1, '2020-07-21': 1, '2020-07-16': 1})
Counter({'2020-08-14': 6, '2020-08-06': 2})
Counter({'2020-07-16': 1, '2020-07-01': 1})
Counter({'2020-08-07': 1, '2020-08-05': 1})
Counter({'2020-08-10': 3, '2020-08-13': 2, '2020-08-12': 1, '2020-08-11': 1, '2020-08-05': 1, '2020-07-31': 1, '2020-07-30': 1})
Counter({'2020-08-10': 1, '2020-08-06': 1, '2020-07-20': 1})
Counter({'2020-07-06': 7, '2020-08-10': 1, '2020-07-20': 1, '2020-07-13': 1})
Counter({'2020-07-06': 7, '2020-08-10': 1, '2020-07-20': 1, '2020-07-13': 1})
Counter({'2020-08-13': 1, '2020-07-30': 1, '2020-07-08': 1})
Counter({'2020-08-13': 1, '2020-07-30': 1, '2020-07-08': 1})
Counter({'2020-07-30': 3, '2020-07-31': 2, '2020-08-07': 1})
Counter({'2020-07-06': 3, '2020-08-10': 2, '2020-08-06': 1, '2020-07-29': 1, '2020-07-10': 1})


In [312]:
df_all.loc['2020-08-14']

ImportError: matplotlib is required for plotting when the default backend "matplotlib" is selected.


## HERE 

In [125]:

con = sqlite3.connect('Stocks.db')
con.execute("CREATE TABLE master(TKR PRIMARY KEY)")


In [152]:
def extractDigits(lst): 
    return list(map(lambda el:[el], lst)) 
      
              
# Driver code 
 
y = extractDigits(x)

y

con.executemany("INSERT  INTO master VALUES (?)", (y))
for row in con.execute("SELECT TKR FROM master"):
    print(row)

con.commit()
con.close()

ProgrammingError: Cannot operate on a closed database.

In [73]:
import pandas as pd 
import csv, sqlite3

#converte the list to df to csv 

df_stocks = pd.DataFrame({'TKR':x})
csv_data = df_stocks.to_csv(r'/Users/layalalzaydi/Desktop/Fillings/Stocks.csv')

connection = sqlite3.connect('Stocks.db')
cursor = connection.cursor()
connection.execute("CREATE TABLE a (TKR TEXT PRIMARY KEY, LINK TEXT);") 

In [64]:
import pandas as pd 
import csv, sqlite3

#converte the list to df to csv 

df_stocks = pd.DataFrame({'TKR':x})
csv_data = df_stocks.to_csv(r'/Users/layalalzaydi/Desktop/Fillings/Stocks.csv')

connection = sqlite3.connect('Stocks.db')
cursor = connection.cursor()
connection.execute("CREATE TABLE t (TKR);") 

with open('Stocks.csv',r'\Users\layalalzaydi\Desktop\Fillings\Stocks.csv') as fin: 
    dr = csv.DictReader(fin) 
    to_db = [(i['TKR']) for i in dr]

cursor.executemany("INSERT INTO t ('TKR') VALUES (?);", to_db)
connection.commit()
connection.close()

ValueError: invalid mode: '\Users\layalalzaydi\Desktop\Fillings\Stocks.csv'